In [2]:
# activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [3]:
#import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
from pyspark import SparkFiles

In [5]:
peterpan_df = spark.read.text('peterpan.txt')

In [6]:
peterpan_df.show()

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
|This eBook is for...|
|    Title: Peter Pan|
| Peter Pan and Wendy|
|Author: James M. ...|
|Release Date: Jul...|
|[Most recently up...|
|   Language: English|
|Character set enc...|
|Produced by: Dunc...|
|*** START OF THE ...|
|           Peter Pan|
|   [PETER AND WENDY]|
|by J. M. Barrie [...|
|A Millennium Fulc...|
|                    |
|            Contents|
|Chapter I. PETER ...|
|Chapter II. THE S...|
|Chapter III. COME...|
+--------------------+
only showing top 20 rows



In [7]:
#Read the input file and calculate words count
from operator import add
peterpan_words = spark.sparkContext.textFile('peterpan.txt')
peterpan_words_split = peterpan_words.flatMap(lambda line: line.split(" "))
peterpan_words_count = peterpan_words_split.map(lambda word: (word, 1))
peterpan_words_count = peterpan_words.reduceByKey(add)

In [1]:
#Save words count to output file
#peterpan_words_count.saveAsTextFile('ppwordcount.txt')
#spark.read.text('ppwordcount.txt').show()

In [7]:
#Read the input file and calculate words count
peterpan_data = spark.sparkContext.textFile('peterpan.txt')
peterpan_data_split = peterpan_data.flatMap(lambda line: line.split(" "))
peterpan_words_count = peterpan_data_split.map(lambda word: (word, 1))
from operator import add
peterpan_all_counts = peterpan_words_count.reduceByKey(add)

#Save words count to output file
peterpan_all_counts.saveAsTextFile('peterpanWordCounts.txt')
spark.read.text('peterpanWordCounts.txt').show()

+-------------------+
|              value|
+-------------------+
|       ('The', 147)|
|    ('Project', 79)|
|        ('of', 919)|
|        ('Pan,', 3)|
|        ('is', 340)|
|        ('use', 19)|
|      ('anyone', 4)|
|    ('anywhere', 2)|
|        ('in', 682)|
|     ('United', 15)|
|      ('States', 7)|
|      ('other', 72)|
|       ('world', 5)|
|        ('at', 318)|
|        ('no', 135)|
|('restrictions', 2)|
| ('whatsoever.', 2)|
|        ('may', 48)|
|        ('it,', 63)|
|       ('give', 26)|
+-------------------+
only showing top 20 rows

